In [8]:
# I can do this block and the next 5, then if I start over, it wipes the memories..
from Agent import Agent

agent_name = "OpenAI"
agent = Agent(agent_name=agent_name)
agent_config = agent.agent_config
embedder, chunk_size = await agent.get_embedder()

In [14]:
# Create a new memory
import os
from hashlib import sha256
from datetime import datetime
from semantic_kernel.memory.memory_record import MemoryRecord
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore
from chromadb.config import Settings

agent_name = "OpenAI"
memories_dir = os.path.join(os.getcwd(), "agents", agent_name, "memories")
chroma_client = ChromaMemoryStore(
    persist_directory=memories_dir,
    client_settings=Settings(
        chroma_db_impl="chromadb.db.duckdb.PersistentDuckDB",  # also tried 'duckdb+parquet' but it did not work at all.
        persist_directory=memories_dir,
        anonymized_telemetry=False,
    ),
)


async def get_memories():
    memories_exist = await chroma_client.does_collection_exist_async("memories")
    print(f"Memories Exist? {memories_exist}")
    if not memories_exist:
        print("Creating memories collection")
        await chroma_client.create_collection_async(collection_name="memories")
        memories = await chroma_client.get_collection_async(collection_name="memories")
    else:
        memories = await chroma_client.get_collection_async(collection_name="memories")
        print(
            "Retrieved memories collection"
        )  # <- This never happens on the first attempt
    return memories


async def new_memory(
    content="Test memory", description="Test memory", external_source_name="Test memory"
):
    await get_memories()
    record = MemoryRecord(
        is_reference=False,
        id=sha256((content + datetime.now().isoformat()).encode()).hexdigest(),
        text=content,
        timestamp=datetime.now().isoformat(),
        description=description,
        external_source_name=external_source_name,  # URL or File path
        embedding=await embedder(content),
    )

    await chroma_client.upsert_async(
        collection_name="memories",
        record=record,
    )


async def get_memory(content="Test memory"):
    # Find a memory
    results = await chroma_client.get_nearest_matches_async(
        collection_name="memories",
        embedding=await embedder(content),
        limit=1,
        min_relevance_score=0.1,
    )
    context = []
    for memory, score in results:
        context.append(memory._text)
    return context


async def memories_exist():
    return await chroma_client.does_collection_exist_async("memories")


2023-05-31 10:12:52,127 | INFO | loaded in 0 embeddings
2023-05-31 10:12:52,128 | INFO | loaded in 0 collections
2023-05-31 10:12:52,128 | INFO | Persisting DB to disk, putting it in the save folder: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories


In [15]:
await memories_exist()

False

In [5]:
await get_memories()


Memories Exist? False
Creating memories collection


Collection(name=memories)

In [12]:
await new_memory(
    description="Test memory", external_source_name="Test memory", content="Test memory"
)


Memories Exist? False
Creating memories collection


2023-05-31 10:12:47,209 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=28 request_id=98c492a56d0aa1422d158106042ca3c7 response_code=200


In [36]:
await get_memory(content="Test memory")

2023-05-31 09:54:15,661 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=143 request_id=d452007715a881e0de32426c2b8f9b7a response_code=200


In [11]:
content = "Test memory"

await agent.memories.context_agent(query=content, top_results_num=2)


AttributeError: 'Memories' object has no attribute 'get_embedder'

In [2]:
# Store a memory
await memories.store_memory(content="Test memory", description="Test memory")


2023-05-31 08:15:38,537 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=260 request_id=ed358be9efe6983fe6b50d7bdd795660 response_code=200


In [4]:
await memories.context_agent(query=content, top_results_num=2)


2023-05-31 08:16:04,802 | INFO | Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
2023-05-31 08:16:04,803 | INFO | Running Chroma using direct local API.
2023-05-31 08:16:04,803 | WARNING | Using embedded DuckDB with persistence: data will be stored in: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories
2023-05-31 08:16:04,806 | INFO | No existing DB found in /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories, skipping load
2023-05-31 08:16:04,806 | INFO | No existing DB found in /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories, skipping load
2023-05-31 08:16:04,807 | INFO | Memories for OpenAI do not exist. Creating...
2023-05-31 08:16:05,417 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=435 request_id=7e2931cce94d959f8849805ff50cfb62 response_code=200
2023-05-31 08:16:05,423 | INFO | Failed to get context: Index not found, please create an instance before querying


[]